# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [4]:
from azureml.core import Workspace, Experiment, Dataset
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core import Environment
from azureml.core import ScriptRunConfig
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform,choice
import os
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.train.automl import AutoMLConfig



## Dataset

### Overview
The dataset to be used is built up from meteorological values for the city of Los Angeles, USA. It includes data like the minimum and maximum temperature on the day, the wind speed and the measured precipitation. 
The objective of this project will be to make a model capable of predicting the precipitation in the city depending on the other given meteorogical measures. For this we will use the autoML tool from Azure as well as a predifined regression model with optimized hyperparameters using HyperDrive.

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [17]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'Capstone_ML'

experiment=Experiment(ws, experiment_name)

dataset = Dataset.get_by_name(ws, name='LA_Rain')


In [16]:
dataset2 = dataset.drop_columns('TAVG')
df =dataset2.to_pandas_dataframe()
df.head()

,STATION,NAME,DATE,AWND,PGTM,PRCP,TMAX,TMIN,WDF2,WDF5,WSF2,WSF5,WT01,WT02,WT08
0,USW00093134,"LOS ANGELES DOWNTOWN USC, CA US",2016-01-01,2.46,None,0.00,64,43,10.0,30.0,8.1,11.0,NaN,NaN,1.0
1,USW00093134,"LOS ANGELES DOWNTOWN USC, CA US",2016-01-02,2.01,None,0.00,65,47,270.0,30.0,6.0,8.9,NaN,NaN,NaN
2,USW00093134,"LOS ANGELES DOWNTOWN USC, CA US",2016-01-03,0.67,None,0.00,62,44,150.0,150.0,10.1,14.1,NaN,NaN,NaN
3,USW00093134,"LOS ANGELES DOWNTOWN USC, CA US",2016-01-04,1.34,None,0.01,69,55,270.0,280.0,8.1,14.1,NaN,NaN,NaN
4,USW00093134,"LOS ANGELES DOWNTOWN USC, CA US",2016-01-05,2.46,None,1.61,59,49,140.0,140.0,10.1,16.1,1.0,1.0,NaN


In [12]:
df.describe()

,AWND,PRCP,TMAX,TMIN,WDF2,WDF5,WSF2,WSF5,WT01,WT02,WT08
count,1823.000000,1827.000000,1827.000000,1827.000000,1822.000000,1809.000000,1823.000000,1809.000000,551.0,39.0,861.0
mean,1.675710,0.035720,76.498632,58.120416,254.143798,256.428966,9.062095,14.721669,1.0,1.0,1.0
std,0.909434,0.189859,9.133929,7.223756,54.756284,55.571902,2.016827,3.622257,0.0,0.0,0.0
min,0.000000,0.000000,52.000000,38.000000,10.000000,10.000000,2.900000,6.000000,1.0,1.0,1.0
25%,1.120000,0.000000,70.000000,53.000000,260.000000,270.000000,8.100000,12.100000,1.0,1.0,1.0
50%,1.570000,0.000000,76.000000,58.000000,270.000000,270.000000,8.900000,14.100000,1.0,1.0,1.0
75%,2.010000,0.000000,83.000000,64.000000,270.000000,280.000000,10.100000,16.100000,1.0,1.0,1.0
max,8.050000,2.670000,111.000000,79.000000,360.000000,360.000000,21.000000,34.000000,1.0,1.0,1.0


### Creating the compute target
Here we will define the compute cluster for our runs

In [7]:
cpu_cluster_name = 'EduardoCluster'
compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',max_nodes=4)
cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [22]:
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'normalized_root_mean_squared_error'
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(compute_target='EduardoCluster',
                             task = "regression",
                             training_data=dataset2,
                             label_column_name="PRCP", 
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

In [23]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config)

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
Capstone_ML,AutoML_27e22871-5b1a-4683-b4ea-b060c049261d,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [24]:
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Best Model

We can retrieve the best model from the autoML run



In [40]:
best_autoML_run, best_autmoML_model = remote_run.get_output()

From the properties we can see that the best model was a Voting Ensemble using the following algorithms:

- 'ExtremeRandomTrees', 
- 'RandomForest'
- 'LightGBM'
-  4 'XGBoostRegressor'
- 'DecisionTree'

In [53]:
best_autoML_run.get_details()

{'runId': 'AutoML_27e22871-5b1a-4683-b4ea-b060c049261d_49',
 'target': 'EduardoCluster',
 'status': 'Completed',
 'startTimeUtc': '2022-01-26T08:23:50.241076Z',
 'endTimeUtc': '2022-01-26T08:25:13.877356Z',
 'services': {},
 'warnings': [{'message': 'This run might be using a new job runtime with improved performance and error reporting. The logs from your script are in user_logs/std_log.txt. Please let us know if you run into any issues, and if you would like to opt-out, please add the environment variable AZUREML_COMPUTE_USE_COMMON_RUNTIME to the environment variables section of the job and set its value to the string "false"'}],
 'properties': {'runTemplate': 'automl_child',
  'pipeline_id': '__AutoML_Ensemble__',
  'pipeline_spec': '{"pipeline_id":"__AutoML_Ensemble__","objects":[{"module":"azureml.train.automl.ensemble","class_name":"Ensemble","spec_class":"sklearn","param_args":[],"param_kwargs":{"automl_settings":"{\'task_type\':\'regression\',\'primary_metric\':\'normalized_roo

We then proceed to register the model for later deployment

In [54]:
#TODO: Save the best model
autoML_model = best_autoML_run.register_model(model_name = 'best_autoML_model.pkl', model_path = './outputs/')

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [76]:
from azureml.core import Model
from azureml.core.conda_dependencies import CondaDependencies


environment = Environment('my-sklearn-environment')
environment.python.conda_dependencies = CondaDependencies.create(pip_packages=[
    'azureml-defaults',
    'inference-schema[numpy-support]',
    'joblib',
    'numpy',
    'scikit-learn=={}'.format(sklearn.__version__)
])

from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice


service_name = 'my-custom-env-service'

inference_config = InferenceConfig(entry_script='score.py', environment=environment)
aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)

service_name = 'precipitation-predictor'
service = Model.deploy(workspace=ws,
                       name=service_name,
                       models=[autoML_model],
                       inference_config=inference_config,
                       deployment_config=aci_config,
                       overwrite=True)


UserErrorException: UserErrorException:
	Message: You must provide an InferenceConfig when deploying a model with model_framework set to Custom. Default environments are only provided for these frameworks: ['Onnx', 'ScikitLearn', 'TensorFlow', 'Multi'].
	InnerException None
	ErrorResponse 
{
    "error": {
        "code": "UserError",
        "message": "You must provide an InferenceConfig when deploying a model with model_framework set to Custom. Default environments are only provided for these frameworks: ['Onnx', 'ScikitLearn', 'TensorFlow', 'Multi']."
    }
}

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
